In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [ ]:
from PIL import Image
from zipfile import ZipFile

#path to data
train_path = os.path.join(dirname, filenames[1])
test_path = os.path.join(dirname, filenames[0])
submission_path = os.path.join(dirname, filenames[2])

In [ ]:
train_data = []
submission = []

#unzip data and prepapre data for training
train_zip = ZipFile(train_path)

file_names = train_zip.namelist()
for i in range(len(file_names)-1):
    zipped_image = train_zip.open(file_names[i+1])
    image = Image.open(zipped_image).resize((64,64))
    name = zipped_image.name.split('.')[-3]
    if name == 'train/cat': train_data.append([np.array(image),  np.array([1,0])])
    elif name == 'train/dog': train_data.append([np.array(image),  np.array([0,1])])
 

train_zip.close()




In [ ]:
train_x = np.zeros((len(train_data), 64,64,3))
train_y = np.zeros((len(train_data),2))
import random
random.shuffle(train_data) # in original data first half are cats and second half are dogs
# so we need to shuffle data before passing it to model
i = 0
for data in train_data:
    train_x[i,] = data[0]
    train_y[i,] = data[1]
    i += 1

In [ ]:
#prepare testing data

test_data = np.zeros((12500,64,64,3))
test_zip = ZipFile(test_path)
test_names = test_zip.namelist()
for i in range(len(test_names)-1):
    zipped_image = test_zip.open(test_names[i+1])
    image = Image.open(zipped_image).resize((64,64))
    test_data[i,] = np.array(image)
test_zip.close()


In [ ]:
# normalize data and center it around zero

mean_pixel = train_x.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = train_x.std(axis=(0, 1, 2), keepdims=True)
train_x = (train_x - mean_pixel) / std_pixel
test_data = (test_data - mean_pixel) / std_pixel

In [ ]:
#extremely simple model in Functional API 

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, AveragePooling2D, BatchNormalization, Dense, Flatten

inputs = tf.keras.layers.Input((64, 64, 3))

conv_layer1 = Conv2D(32, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(inputs)
batch_norm1 = BatchNormalization()(conv_layer1)
conv_layer2 = Conv2D(32, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(batch_norm1)
#batch_norm2 = BatchNormalization()(conv_layer2)
max_pool1 = MaxPooling2D(strides = 2)(conv_layer2) # 32x32x32


conv_layer3 = Conv2D(64, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(max_pool1)
batch_norm3 = BatchNormalization()(conv_layer3)
conv_layer4 = Conv2D(64, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(batch_norm3)
#batch_norm4 = BatchNormalization()(conv_layer4)
max_pool2 = MaxPooling2D(strides = 2)(conv_layer4) # 16x16x64


conv_layer5 = Conv2D(128, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(max_pool2)
batch_norm5 = BatchNormalization()(conv_layer5)
conv_layer6 = Conv2D(128, 3, activation = 'relu', kernel_initializer = 'he_normal',
                    padding = 'same', kernel_regularizer = 'l2')(batch_norm5)
#batch_norm6 = BatchNormalization()(conv_layer6)
max_pool3 = MaxPooling2D(strides = 2)(conv_layer6) # 8x8x128

average_pool = AveragePooling2D(pool_size = (8,8), strides = 1)(max_pool3) # using average pooling will significantly reduce training parameters
dense1 = Dense(150, activation = 'relu', kernel_regularizer = 'l2')(average_pool)
flattened = Flatten()(dense1)
dense2 = Dense(2, activation = 'softmax')(flattened)

simple_model = tf.keras.Model(inputs = inputs, outputs = dense2)
simple_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
simple_model.summary()




Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_42 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_43 (Batc (None, 32, 32, 64)        256 

In [ ]:
results = simple_model.fit(train_x, train_y, validation_split=0.2, batch_size=128, epochs=20)

Epoch 1/20
157/157 [==============================] - 6s 31ms/step - loss: 6.4228 - accuracy: 0.6393 - val_loss: 1.6389 - val_accuracy: 0.5116
Epoch 2/20
157/157 [==============================] - 4s 27ms/step - loss: 1.2542 - accuracy: 0.7480 - val_loss: 1.3223 - val_accuracy: 0.5116
Epoch 3/20
157/157 [==============================] - 4s 28ms/step - loss: 0.8256 - accuracy: 0.7914 - val_loss: 0.8214 - val_accuracy: 0.7316
Epoch 4/20
157/157 [==============================] - 4s 28ms/step - loss: 0.6849 - accuracy: 0.8210 - val_loss: 0.7302 - val_accuracy: 0.7728
Epoch 5/20
157/157 [==============================] - 4s 27ms/step - loss: 0.6128 - accuracy: 0.8335 - val_loss: 0.6700 - val_accuracy: 0.7964
Epoch 6/20
157/157 [==============================] - 4s 28ms/step - loss: 0.5496 - accuracy: 0.8561 - val_loss: 0.7445 - val_accuracy: 0.7070
Epoch 7/20
157/157 [==============================] - 4s 28ms/step - loss: 0.5222 - accuracy: 0.8597 - val_loss: 0.7012 - val_accuracy: 0.7440